<a href="https://colab.research.google.com/github/jmhuer/shift_invariant_dictionary_learning/blob/main/pop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pretty_midi
!git clone https://github.com/music-x-lab/POP909-Dataset
%cd /content/POP909-Dataset/data_process


     |████████████████████████████████| 5.6 MB 6.7 MB/s 
     |████████████████████████████████| 51 kB 6.9 MB/s 
  Created wheel for pretty-midi: filename=pretty_midi-0.2.9-py3-none-any.whl size=5591953 sha256=949c75f7dda18e7a8baf1ad4ba85b386e853242a6d690f01baac34f2aabf5dd6
  Stored in directory: /root/.cache/pip/wheels/ad/74/7c/a06473ca8dcb63efb98c1e67667ce39d52100f837835ea18fa
Successfully built pretty-midi
Cloning into 'POP909-Dataset'...
remote: Enumerating objects: 9265, done.
remote: Counting objects: 100% (9265/9265), done.
remote: Compressing objects: 100% (8157/8157), done.
remote: Total 9265 (delta 13), reused 9245 (delta 6), pack-reused 0
Receiving objects: 100% (9265/9265), 45.75 MiB | 29.62 MiB/s, done.
Resolving deltas: 100% (13/13), done.
/content/POP909-Dataset/data_process


In [2]:
#@title Pytorch for DL

import torch.nn.functional as F
import torch.optim as optim
from torch import nn
import torch
from torch.nn.utils import weight_norm
import numpy as np

def get_model_parameters(model):
    model_parameters = filter(lambda p: p.requires_grad, model.parameters())
    params = sum([np.prod(p.size()) for p in model_parameters])
    return params

In [3]:

import pickle
import os
import sys
# import utils
from processor import MidiEventProcessor
import pretty_midi 
import numpy as np


total = 0
def preprocess_midi(path):
    global total
    data = pretty_midi.PrettyMIDI(path)
    main_notes = []
    acc_notes = []
    for ins in data.instruments:
        acc_notes.extend(ins.notes)
    for i in range(len(main_notes)):
        main_notes[i].start = round(main_notes[i].start,2)
        main_notes[i].end = round(main_notes[i].end,2)
    for i in range(len(acc_notes)):
        acc_notes[i].start = round(acc_notes[i].start,2)
        acc_notes[i].end = round(acc_notes[i].end,2)
    main_notes.sort(key = lambda x:x.start)
    acc_notes.sort(key = lambda x:x.start)

    piano_program = pretty_midi.instrument_name_to_program('Acoustic Grand Piano')
    piano = pretty_midi.Instrument(program=piano_program)
    piano.notes.extend(acc_notes)
    # score = prettyn
    # mpr = MidiEventProcessor()
    # repr_seq = mpr.encode([main_notes, acc_notes])
    total += 1
    return piano.get_piano_roll()

def preprocess_pop909(midi_root, save_dir):
    save_py = []
    midi_paths = [d for d in os.listdir(midi_root)] #not index
    i = 0
    out_fmt = '{}-{}.data'
    for path in midi_paths:
        if "." not in path:
          # print(' ', end='[{}]'.format(path), flush=True)
          filename = midi_root + path  +"/"+path[0:3] + ".mid"
          try:
              data = torch.tensor(preprocess_midi(filename))
              print(data.shape)
          except KeyboardInterrupt:
              print(' Abort')
              return
          except EOFError:
              print('EOF Error')
              return
          save_py.append(data)
    return save_py
     
    
# replace the folder with your POP909 data folder
train_dataset = preprocess_pop909("../POP909/","midi_data/")



torch.Size([128, 26426])
torch.Size([128, 16903])
torch.Size([128, 21515])
torch.Size([128, 24289])
torch.Size([128, 27367])
torch.Size([128, 23618])
torch.Size([128, 24140])
torch.Size([128, 25326])
torch.Size([128, 25852])
torch.Size([128, 27320])
torch.Size([128, 28448])
torch.Size([128, 29813])
torch.Size([128, 22922])
torch.Size([128, 23535])
torch.Size([128, 24609])
torch.Size([128, 25240])
torch.Size([128, 21341])
torch.Size([128, 25675])
torch.Size([128, 23753])
torch.Size([128, 24925])
torch.Size([128, 25786])
torch.Size([128, 27589])
torch.Size([128, 22386])
torch.Size([128, 28606])
torch.Size([128, 25860])
torch.Size([128, 25355])
torch.Size([128, 32616])
torch.Size([128, 31954])
torch.Size([128, 27881])
torch.Size([128, 22469])
torch.Size([128, 22353])
torch.Size([128, 25545])
torch.Size([128, 27894])
torch.Size([128, 25400])
torch.Size([128, 30304])
torch.Size([128, 25229])
torch.Size([128, 21738])
torch.Size([128, 23334])
torch.Size([128, 26036])
torch.Size([128, 14997])


In [4]:
import IPython.display

# cello_c_chord.write('cello-C-chord.mid')
pm = piano_roll_to_pretty_midi(train_dataset[2])
IPython.display.Audio(pm.synthesize(fs=16000), rate=16000)



NameError: ignored

In [ ]:


def piano_roll_to_pretty_midi(piano_roll, fs=100, program=0):
    '''Convert a Piano Roll array into a PrettyMidi object
     with a single instrument.
    Parameters
    ----------
    piano_roll : np.ndarray, shape=(128,frames), dtype=int
        Piano roll of one instrument
    fs : int
        Sampling frequency of the columns, i.e. each column is spaced apart
        by ``1./fs`` seconds.
    program : int
        The program number of the instrument.
    Returns
    -------
    midi_object : pretty_midi.PrettyMIDI
        A pretty_midi.PrettyMIDI class instance describing
        the piano roll.
    '''
    notes, frames = piano_roll.shape
    pm = pretty_midi.PrettyMIDI()
    instrument = pretty_midi.Instrument(program=program)

    # pad 1 column of zeros so we can acknowledge inital and ending events
    piano_roll = np.pad(piano_roll, [(0, 0), (1, 1)], 'constant')

    # use changes in velocities to find note on / note off events
    velocity_changes = np.nonzero(np.diff(piano_roll).T)

    # keep track on velocities and note on times
    prev_velocities = np.zeros(notes, dtype=int)
    note_on_time = np.zeros(notes)

    for time, note in zip(*velocity_changes):
        # use time + 1 because of padding above
        velocity = piano_roll[note, time + 1]
        time = time / fs
        if velocity > 0:
            if prev_velocities[note] == 0:
                note_on_time[note] = time
                prev_velocities[note] = velocity
        else:
            pm_note = pretty_midi.Note(
                velocity=prev_velocities[note],
                pitch=note,
                start=note_on_time[note],
                end=time)
            instrument.notes.append(pm_note)
            prev_velocities[note] = 0
    pm.instruments.append(instrument)
    return pm




#were in bussiness

In [ ]:
#@title KWTA


class SparsifyBase(nn.Module):
    def __init__(self, sparse_ratio=0.5):
        super(SparsifyBase, self).__init__()
        self.sr = sparse_ratio
        self.preact = None
        self.act = None
    def get_activation(self):
        def hook(model, input, output):
            self.preact = input[0].cpu().detach().clone()
            self.act = output.cpu().detach().clone()
        return hook
    def record_activation(self):
        self.register_forward_hook(self.get_activation())


class Sparsify1D_kactive(SparsifyBase):
    def __init__(self, k=1):
        super(Sparsify1D_kactive, self).__init__()
        self.k = k
    def forward(self, x):
        m = torch.zeros(x.shape).to(device)
        for i in range(self.k):
            indeces = x.topk(self.k, dim=1)[1][:, i]
            m += torch.mul(torch.zeros(x.shape).to(device).scatter(1, indeces.unsqueeze(1), 1), x)
            # print("\n hi", m )
        return m.double()

In [ ]:
#@title TCN 

class Chomp1d(nn.Module):
    def __init__(self, chomp_size):
        super(Chomp1d, self).__init__()
        self.chomp_size = chomp_size

    def forward(self, x):
        return x[:, :, :-self.chomp_size].contiguous()


class TemporalBlock(nn.Module):
    def __init__(self, n_inputs, n_outputs, kernel_size, stride, dilation, padding, dropout=0.2):
        super(TemporalBlock, self).__init__()
        self.conv1 = weight_norm(nn.Conv1d(n_inputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation))
        self.chomp1 = Chomp1d(padding)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout)

        self.conv2 = weight_norm(nn.Conv1d(n_outputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation))
        self.chomp2 = Chomp1d(padding)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout)

        self.net = nn.Sequential(self.conv1, self.chomp1, self.relu1, self.dropout1,
                                 self.conv2, self.chomp2, self.relu2, self.dropout2)
        self.downsample = nn.Conv1d(n_inputs, n_outputs, 1) if n_inputs != n_outputs else None
        self.relu = nn.ReLU()
        self.init_weights()

    def init_weights(self):
        self.conv1.weight.data.normal_(0, 1)
        self.conv2.weight.data.normal_(0, 1)
        if self.downsample is not None:
            self.downsample.weight.data.normal_(0, 1)

    def forward(self, x):
        # print("block ", x.size())
        out = self.net(x)
        res = x if self.downsample is None else self.downsample(x)
        return self.relu(out + res)


class TemporalConvNet(nn.Module):
    def __init__(self, num_inputs, num_channels, kernel_size=2, dropout=0.2):
        super(TemporalConvNet, self).__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_channels = num_inputs if i == 0 else num_channels[i-1]
            out_channels = num_channels[i]
            layers += [TemporalBlock(in_channels, out_channels, kernel_size, stride=1, dilation=dilation_size,
                                     padding=(kernel_size-1) * dilation_size, dropout=dropout)]

        self.network = nn.Sequential(*layers)


        # print("last layer conv", self.network[-1].conv2.weight.data[:,0,:].size())
        # print("last layer conv", self.network[-1].conv2.weight.data[:,0,:])

    def forward(self, x):
        return self.network(x)


In [ ]:
#@title TCN - Autoeconder 

class TCNAutoencoder(nn.Module):
    def __init__(self, kernel_size, dropout, wta_k):
        super(TCNAutoencoder, self).__init__()
        self.wta = Sparsify1D_kactive(k = wta_k)
        self.feature = TemporalConvNet(128, [128*2,128*4], kernel_size, dropout=dropout).double()
        self.encoder = torch.nn.Conv1d(in_channels=128*4, out_channels=200, kernel_size=kernel_size, padding=0, bias=True, stride=4)
        self.decoder = torch.nn.ConvTranspose1d(in_channels=200, out_channels=1, kernel_size=kernel_size, padding=0, bias=True, stride=4)
        # self.encoder.weight.data.normal_(30)
        # self.decoder.weight.data.normal_(300)
        self.relu1 = nn.ReLU()
        self.code = None
        # torch.nn.init.xavier_uniform(self.encoder.weight)
        # torch.nn.init.xavier_uniform(self.decoder.weight)
    def get_kernels(self):
        return self.decoder.weight.data[:,0,:]
    def feature_map(self, x):
        code = self.code
        return code
    def forward(self, x):
        # x needs to have dimension (N, C, L) in order to be passed into CNN
        output = self.feature(x)
        self.code = self.wta(self.encoder(output))
        output = self.decoder(self.code )
        return output


In [ ]:
#@title GO
train_dataset = data 
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print("Using device: ", device)

model = TCNAutoencoder(kernel_size=4, 
                       dropout=0.2, 
                       wta_k = 95).to(device).double()
print("TCNAutoencoder trainable parameters: ", get_model_parameters(model))

# model.load_state_dict(torch.load("model.pth"))


loss_fn = torch.nn.MSELoss().to(device)
# optimizer = optim.SGD(model.parameters(), lr=.01, weight_decay = 0.00001, momentum=0.05) ##this has weight decay just like you implemented
optimizer = optim.AdamW(model.parameters(), lr=.005,  betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=True) ##this has weight decay just like you implemented
epochs = 60
history = {"loss": []}

calc = []
total_len = 0
for i in range(epochs):
    #decaying WTA
    if i % 12 == 0 and i != 0:
        model.wta.k = max(1, model.wta.k - 5)
        print("model.wta.k: ", model.wta.k)
    for train_data in train_dataset:
        # calc.extend(train_data.flatten().numpy())
        #normalize 
        # train_data = (train_data - 224.15541543187527) / 111.14747885919755
        #preprocess
        lenby4 = len(train_data) // 4
        train_data = train_data[None, None, 0:lenby4*4].to(device).double()
        
        #preprocess
        optimizer.zero_grad()
        output = model(train_data)

        loss = loss_fn(output, train_data)
        loss.backward()
        # torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
        optimizer.step()
        history["loss"].append(float(loss)*lenby4)
        total_len += lenby4
    print("Epoch : {} \t Loss : {} ".format(i, round(float(np.mean(history["loss"], axis=0)/total_len),7)))
    history["loss"] = []
    total_len = 0

# print(len(calc))
# print(np.mean(calc, axis=0))
# print(np.std(calc, axis=0))
